# Eventbus

## Events

Events are json serializable objects with a mandatory topic field. Convention for request/response events:

| Message  | Topic    |
| :------- | :------: |
| Request  | `?topic` |
| Response | `!topic` |
| Error    | `#topic` |

**Examples:**

```python
request_event = { "topic": "?state" }
response_event = { "topic": "!state", "state": "on" }
```

## Bus

The eventbus provides methods to emit and listen to events.

In [2]:
from eventbus import bus

bus.unsubscribe_all()

@bus.on("*")
def all_events(**event):
    print("Received event", event)

@bus.on("?state")
async def request_state(**event):   
    await bus.emit(topic="!state", state="on", eid="a-b-c")

@bus.on("!state")
async def response_state(topic, src, state, eid):
    print(f"Received update for entity {eid}: state = {state}")

await bus.emit(topic="?state")

Received update for entity a-b-c: state = on
Received event {'topic': '!state', 'state': 'on', 'eid': 'a-b-c', 'src': 'leaf_id'}
Received event {'topic': '?state', 'src': 'leaf_id'}


Sending events from synchronous code:

In [3]:
import asyncio
from eventbus import bus

def cb(**event):
    print("Got *", event)

bus.subscribe(cb, "*")

bus.emit_sync(topic="sync_event")

# wait for sync event to be processed
await asyncio.sleep(0.1)

# async events are handled immediately
await bus.emit(topic="async_event")

bus.unsubscribe(cb, "*")

print("unsubscribed")

@bus.on("other_event")
def sync_event(**event):
    print("Got other_event", event)

bus.emit_sync(topic="other_event")

Received event {'topic': 'sync_event', 'src': 'leaf_id'}
Got * {'topic': 'sync_event', 'src': 'leaf_id'}
Received event {'topic': 'async_event', 'src': 'leaf_id'}
Got * {'topic': 'async_event', 'src': 'leaf_id'}
unsubscribed
Got other_event {'topic': 'other_event', 'src': 'leaf_id'}
Received event {'topic': 'other_event', 'src': 'leaf_id'}
